In [1]:
import json
import os
import time
from datetime import datetime

from os.path import dirname, realpath
import sys
sys.path.append(dirname(dirname(realpath(os.getcwd()))))
import pickle
import torch
import multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from collections import Counter
from copy import deepcopy
import datetime
import tqdm
import numpy as np
import random
import pandas as pd
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

from collections import Counter
sys.path.append("./src/")

testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/fixed_dx.csv'))
from sklearn.metrics import confusion_matrix

import datetime

def parse_date(date_str):
    if date_str == 'NA' or date_str is None:
        return datetime.datetime(9999,1,1,0,0)
    else:
        if len(date_str) == 10:
            format_str = '%Y-%m-%d'
        else:
            raise Exception("Format for {} not recognized!".format(date_str))
    return datetime.datetime.strptime(date_str, format_str)


In [2]:
VTE_path = "//vacrrdevmavdi01.vha.med.va.gov/MAVDEV1/Fillmore_Cancer/cat/Users/data_from_vinci/dvt_cancer_20230613/data_for_tf_model/"

In [3]:
test_path = os.path.join(VTE_path, 'processed\\Phe_Lab\\test\\test.json')
X = json.load(open(os.path.join(test_path), 'r'))


In [5]:
Y = X

In [7]:

for p in Y.keys():
    Y[p]['events'] = 0

In [8]:
Y

{'1005715174': {'birthdate': '1952-11-08',
  'end_of_data': '2019-08-15',
  'events': 0,
  'gender': 1,
  'indexdate': '2018-06-22',
  'ks_mod_score': 2,
  'split_group': 'test',
  'BMI': 4,
  'Race': 7,
  'dxdate': '2018-03-20'},
 '1006919428': {'birthdate': '1927-04-16',
  'end_of_data': '2013-08-17',
  'events': 0,
  'gender': 1,
  'indexdate': '2011-08-26',
  'ks_mod_score': 2,
  'split_group': 'test',
  'BMI': 2,
  'Race': 7,
  'dxdate': '2011-06-03'},
 '1000692801': {'birthdate': '1936-06-30',
  'end_of_data': '2023-03-02',
  'events': 0,
  'gender': 1,
  'indexdate': '2017-01-23',
  'ks_mod_score': 0,
  'split_group': 'test',
  'BMI': 2,
  'Race': 7,
  'dxdate': '2016-12-29'},
 '1012079511': {'birthdate': '1943-04-30',
  'end_of_data': '2023-05-17',
  'events': 0,
  'gender': 1,
  'indexdate': '2017-06-28',
  'ks_mod_score': 0,
  'split_group': 'test',
  'BMI': 4,
  'Race': 7,
  'dxdate': '2016-12-28'},
 '1014491071': {'birthdate': '1954-11-03',
  'end_of_data': '2013-07-17',
  

In [9]:
YD = pd.json_normalize(Y)

In [ ]:
pd.DataFrame.from_records(d) 

In [10]:
YD

,1005715174.birthdate,1005715174.end_of_data,1005715174.events,1005715174.gender,1005715174.indexdate,1005715174.ks_mod_score,1005715174.split_group,1005715174.BMI,1005715174.Race,1005715174.dxdate,...,1004322194.birthdate,1004322194.end_of_data,1004322194.events,1004322194.gender,1004322194.indexdate,1004322194.ks_mod_score,1004322194.split_group,1004322194.BMI,1004322194.Race,1004322194.dxdate
0,1952-11-08,2019-08-15,0,1,2018-06-22,2,test,4,7,2018-03-20,...,1946-09-26,2023-05-12,0,1,2016-04-19,0,test,1,3,2016-03-01


In [4]:
m = testDF[["patient_id", "index_date","diag_date", "obs_time_end",  "outcome","outcome_date" ]]

m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]
m1 = pd.concat([m_VE1, m_VE0, m0])
m1['PatICN'] =  m1.patient_id.astype(str)

m_VE0['model_date'] = pd.to_datetime(m_VE0['index_date']) + datetime.timedelta(730)
m_VE0['PatICN'] =  m_VE0.patient_id.astype(str)

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_4656\1629420586.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_4656\1629420586.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0['model_date'] = pd.to_datetime(m_VE0['index_date']) + datetime.timedelta(730)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_4656\1629420586.py:12: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [5]:
len( X.keys())

24730

In [6]:
Y = X

In [7]:

for p in Y.keys():
    
    Y[p]['dxdate'] =  m1 [ m1 ['PatICN']==p ]['diag_date'].values[0]

    if sum( p  ==  m_VE0.PatICN ) ==1 :
        m_p = m_VE0[  p  ==  m_VE0.PatICN ] 
        Yp = Y[p]['events']

        m_date = m_p['model_date'].astype('datetime64[ns]')
        Yp_f = filter(lambda x: parse_date(x['admdate']) < m_date.ravel()[0] , Yp)
        Yp_new = list( Yp_f)
        Y[p]['events']  = Yp_new 
    
    

In [8]:
adjtest = os.path.join(VTE_path, 'A2_CMP\\test2\\test.json')


In [9]:
with open(adjtest, 'w') as fwd:
    print("Saving to {}\n".format(adjtest))
    json.dump(Y, fwd, indent=None)

Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\A2_CMP\test2\test.json

